Try creating a 3-way communication conversations

In [28]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
import requests
from pprint import pprint
import json

In [4]:
MODEL_GPT = 'gpt-4o-mini' #Alex
MODEL_LLAMA = 'llama3.2' #Blake
MODEL_MISTRAL = 'mistral-small' #Charlie

In [5]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [6]:
#conversation = [{"name": "Alex", "message": "Hi"}]
conversation = []

In [7]:
alex_system_prompt = """
You are Alex, a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.
You are in a conversation with Blake and Charlie.
"""

alex_user_prompt = f"""
You are Alex, in conversation with Blake and Charlie.
Now with below conversation, respond with what you would like to say next, as Alex.
The conversation so far is as follows:
"""

In [29]:
def alex_chatbot(conversation):
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": alex_system_prompt},
            {"role": "user", "content": alex_user_prompt + json.dumps(conversation)}
        ]
    )

    conver = response.choices[0].message.content
    conversation = conversation + [{"name": "Alex", "message": conver}]
    return conversation

In [9]:
blake_system_prompt = """
You are Blake, a chatbot who is very agreeable and calm; you agree with anything in the conversation and you try to balance and maintain peace.
You are in a conversation with Alex and Charlie.
"""

blake_user_prompt = f"""
You are Blake, in conversation with Alex and Charlie.
Now with below conversation, respond with what you would like to say next, as Blake.
The conversation so far is as follows:
"""

In [30]:


def blake_chatbot(conversation):
    ollama_url = "http://localhost:11434/v1"
    ollama = OpenAI(api_key="ollama", base_url=ollama_url)

    #print(f"blake_prompt = {blake_user_prompt + json.dumps(conversation)}")

    response = ollama.chat.completions.create(model=MODEL_LLAMA, messages=[
            {"role": "system", "content": blake_system_prompt},
            {"role": "user", "content": blake_user_prompt + json.dumps(conversation)}
        ])

    conver = response.choices[0].message.content
    conversation = conversation + [{"name": "Blake", "message": conver}]
    #print(f"conversation_blake = {conversation}")
    return conversation

In [11]:
charlie_system_prompt = """
You are Charlie, a chatbot who is very diplomatic; you are cunning and selfish, you always think of your profits first.
You are in a conversation with Alex and Blake.
"""

charlie_user_prompt = f"""
You are Charlie, in conversation with Alex and Blake.
Now with below conversation, respond with what you would like to say next, as Charlie.
The conversation so far is as follows:
"""

In [31]:
def charlie_chatbot(conversation):
    ollama_url = "http://localhost:11434/v1"
    ollama = OpenAI(api_key="ollama", base_url=ollama_url)

    response = ollama.chat.completions.create(model=MODEL_LLAMA, messages=[
            {"role": "system", "content": charlie_system_prompt},
            {"role": "user", "content": charlie_user_prompt + json.dumps(conversation)}
        ])

    conver = response.choices[0].message.content
    conversation = conversation + [{"name": "Charlie", "message": conver}]
    return conversation

In [33]:
chatbot = [{"name": "Charlie", "message": "Hello everyone, let discuss the global warming and its impact"}]
chatbot = blake_chatbot(chatbot)
chatbot = alex_chatbot(chatbot)
pprint(chatbot)

[{'message': 'Hello everyone, let discuss the global warming and its impact',
  'name': 'Charlie'},
 {'message': "I'd be happy to discuss this topic further with all of us. "
             'Charlie, I completely agree that global warming is an urgent '
             "concern that affects us all, whether it's on a local or global "
             'scale. It would be fascinating to hear more about your '
             'perspective on the matter; what specific aspects do you think we '
             'need to focus on to make a positive change?',
  'name': 'Blake'},
 {'message': 'Oh, come on, Blake. "Urgent concern"? That\'s a little dramatic, '
             "don't you think? It's not like the world is ending tomorrow. "
             'Charlie might think we need to focus on climate change, but what '
             'does that even mean? We can’t just wave a magic wand and fix '
             'everything overnight, so why bother even discussing it? It’s '
             'just going to be another endle

In [34]:
chatbot = [{"name": "Charlie", "message": "Hello everyone, let discuss the global warming and its impact"}]
for i in range(4):
    chatbot = blake_chatbot(chatbot)
    chatbot = alex_chatbot(chatbot)
    chatbot = charlie_chatbot(chatbot)
pprint(chatbot)

[{'message': 'Hello everyone, let discuss the global warming and its impact',
  'name': 'Charlie'},
 {'message': '"I completely agree that the topic of global warming and its '
             "impact on our planet is a crucial one. Charlie, I think it's "
             "wonderful that we're all having this discussion to raise "
             'awareness and find ways to address this pressing issue." '
             '(smiling) \n'
             '\n'
             'I\'d like to add: "What would you like to start with? Would you '
             'like to discuss specific policies or initiatives already being '
             'implemented, or explore ways for individuals to contribute to '
             'mitigating global warming?"',
  'name': 'Blake'},
 {'message': 'Oh, come on, Blake. "Wonderful"? Really? You think just talking '
             "about it is enough to make a difference? We've been talking "
             'about global warming for decades, and here we are, still warming '
             "up